<h1 div class='alert alert-success'><center> TPS Feb 2022 - Ponto de partida (EDA)</center></h1>

![](https://storage.googleapis.com/kaggle-competitions/kaggle/26480/logos/header.png?t=2021-04-09-00-57-05)

# <div class="alert alert-success">  OBJETIVO </div> 

Neste notebook vamos fazer uma análise (EDA) para conhencer os dados dos datasets e estabelecer uma linha de base.


---

# <div class="alert alert-success">  1. IMPORTAÇÕES </div> 

## 1.1. Instalações

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

## 1.2. Bibliotecas 

In [ ]:
import warnings
import random
import os
import gc
import torch
import sklearn.exceptions
import re

In [ ]:
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt 
import seaborn           as sns
import joblib            as jb
import xgboost           as xgb

In [ ]:
from sklearn.model_selection import train_test_split,  KFold, StratifiedKFold
from sklearn.preprocessing   import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing   import MaxAbsScaler, QuantileTransformer, LabelEncoder,normalize
from sklearn.impute          import SimpleImputer
from sklearn                 import metrics
from datetime                import datetime
from scipy.stats             import mode

In [ ]:
from sklearn.utils.class_weight import compute_sample_weight

## 1.3. Funções
Aqui centralizamos todas as funções desenvolvidas durante o projeto para melhor organização do código.

In [ ]:
def jupyter_setting():
    
    %matplotlib inline
      
    #os.environ["WANDB_SILENT"] = "true" 
    #plt.style.use('bmh') 
    #plt.rcParams['figure.figsize'] = [20,15]
    #plt.rcParams['font.size']      = 13
     
    pd.options.display.max_columns = None
    #pd.set_option('display.expand_frame_repr', False)

    warnings.filterwarnings(action='ignore')
    warnings.simplefilter('ignore')
    warnings.filterwarnings('ignore')
    warnings.filterwarnings('ignore', category=DeprecationWarning)
    warnings.filterwarnings('ignore', category=FutureWarning)
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    warnings.filterwarnings('ignore', category=UserWarning)
    warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)
    warnings.filterwarnings("ignore", category= sklearn.exceptions.UndefinedMetricWarning)

    pd.set_option('display.max_rows', 200)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.max_colwidth', None)

    icecream = ["#00008b", "#960018","#008b00", "#00468b", "#8b4500", "#582c00"]
    #sns.palplot(sns.color_palette(icecream))
    
    colors = ["lightcoral", "sandybrown", "darkorange", "mediumseagreen",
          "lightseagreen", "cornflowerblue", "mediumpurple", "palevioletred",
          "lightskyblue", "sandybrown", "yellowgreen", "indianred",
          "lightsteelblue", "mediumorchid", "deepskyblue"]
    
    # Colors
    dark_red   = "#b20710"
    black      = "#221f1f"
    green      = "#009473"
    myred      = '#CD5C5C'
    myblue     = '#6495ED'
    mygreen    = '#90EE90'    
    color_cols = [myred, myblue,mygreen]
    
    return icecream, colors, color_cols

icecream, colors, color_cols = jupyter_setting()

In [ ]:
def missing_zero_values_table(df):
        mis_val         = df.isnull().sum()
        mis_val_percent = round(df.isnull().mean().mul(100), 2)
        mz_table        = pd.concat([mis_val, mis_val_percent], axis=1)
        mz_table        = mz_table.rename(columns = {df.index.name:'col_name', 
                                                     0 : 'Valores ausentes', 
                                                     1 : '% de valores totais'})
        
        mz_table['Tipo de dados'] = df.dtypes
        mz_table                  = mz_table[mz_table.iloc[:,1] != 0 ]. \
                                     sort_values('% de valores totais', ascending=False)
        
        msg = "Seu dataframe selecionado tem {} colunas e {} " + \
              "linhas. \nExistem {} colunas com valores ausentes."
            
        print (msg.format(df.shape[1], df.shape[0], mz_table.shape[0]))
        
        return mz_table.reset_index()

In [ ]:
def reduce_memory_usage(df, verbose=True):
    
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    
    for col in df.columns:
        
        col_type = df[col].dtypes
        
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
        
    return df

In [ ]:
def graf_feature_corr(df, annot_=False, threshold=.8, print_var=False):
    
    df = df.corr(method ='pearson').round(5)

    # Máscara para ocultar a parte superior direita do gráfico, pois é uma duplicata
    mask = np.zeros_like(df)
    mask[np.triu_indices_from(mask)] = True

    # Making a plot
    plt.figure(figsize=(20,12))
    ax = sns.heatmap(df, annot=annot_, mask=mask, cmap="RdBu", annot_kws={"weight": "bold", "fontsize":13})

    ax.set_title("Mapa de calor de correlação das variável", fontsize=17)

    plt.setp(ax.get_xticklabels(), 
             rotation      = 90, 
             ha            = "right",
             rotation_mode = "anchor", 
             weight        = "normal")

    plt.setp(ax.get_yticklabels(), 
             weight        = "normal",
             rotation_mode = "anchor", 
             rotation      = 0, 
             ha            = "right");
    
    if print_var: 
        print('Variáveis autocorrelacionadas threshold={:2.2f}'.format(threshold))
        df_corr = df[abs(df)>threshold][df!=1.0].unstack().dropna().reset_index()
        df_corr.columns =  ['var_1', 'var_2', 'corr']
        display(df_corr)

In [ ]:
def free_gpu_cache():
    
    # https://www.kaggle.com/getting-started/140636
    #print("Initial GPU Usage")
    #gpu_usage()                             

    #cuda.select_device(0)
    #cuda.close()
    #cuda.select_device(0)   
    
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
def diff(t_a, t_b):
    from dateutil.relativedelta import relativedelta
    t_diff = relativedelta(t_b, t_a)  # later/end time comes first!
    return '{h}h {m}m {s}s'.format(h=t_diff.hours, m=t_diff.minutes, s=t_diff.seconds)

In [ ]:
def describe(df):
    var = df.columns

    # Medidas de tendência central, média e mediana 
    ct1 = pd.DataFrame(df[var].apply(np.mean)).T
    ct2 = pd.DataFrame(df[var].apply(np.median)).T

    # Dispensão - str, min , max range skew, kurtosis
    d1 = pd.DataFrame(df[var].apply(np.std)).T
    d2 = pd.DataFrame(df[var].apply(min)).T
    d3 = pd.DataFrame(df[var].apply(max)).T
    d4 = pd.DataFrame(df[var].apply(lambda x: x.max() - x.min())).T
    d5 = pd.DataFrame(df[var].apply(lambda x: x.skew())).T
    d6 = pd.DataFrame(df[var].apply(lambda x: x.kurtosis())).T
    d7 = pd.DataFrame(df[var].apply(lambda x: (3 *( np.mean(x) - np.median(x)) / np.std(x) ))).T

    # concatenete 
    m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6, d7]).T.reset_index()
    m.columns = ['attrobutes', 'min', 'max', 'range', 'mean', 'median', 'std','skew', 'kurtosis','coef_as']
    
    return m

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    fig, ax = plt.subplots()
    ax.plot(fpr, tpr, "r-", label=label)
    ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", c=".3")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.rcParams['font.size'] = 12
    plt.title('ROC curve for TPS 09')
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity)')
    plt.legend(loc="lower right")
    plt.grid(True)

In [ ]:
def confusion_plot(matrix, labels = None, title = None):
        
    labels = labels if labels else ['Negative (0)', 'Positive (1)']    
    
    fig, ax = plt.subplots(nrows=1, ncols=1)
    
    sns.heatmap(data        = matrix, 
                cmap        = 'Blues', 
                annot       = True, 
                fmt         = 'd',
                xticklabels = labels, 
                yticklabels = labels, 
                ax          = ax);
    
    ax.set_xlabel('\n PREVISTO', fontsize=15)
    ax.set_ylabel('REAL \n', fontsize=15)
    ax.set_title(title)
    
    plt.close();
    
    return fig;

In [ ]:
def graf_outlier(df, feature):
    col = [(0,4), (5,9)]

    df_plot = ((df[feature] - df[feature].min())/
               (df[feature].max() - df[feature].min()))

    fig, ax = plt.subplots(len(col), 1, figsize=(15,7))

    for i, (x) in enumerate(col): 
        sns.boxplot(data = df_plot.iloc[:, x[0]:x[1] ], ax = ax[i]); 

In [ ]:
def graf_eval():

    results     = model.evals_result()
    ntree_limit = model.best_ntree_limit

    plt.figure(figsize=(20,7))

    for i, error in  enumerate(['mlogloss', 'merror']):#
        
        plt.subplot(1,2,i+1)
        plt.plot(results["validation_0"][error], label="Treinamento")
        plt.plot(results["validation_1"][error], label="Validação")

        plt.axvline(ntree_limit, 
                    color="gray", 
                    label="N. de árvore ideal {}".format(ntree_limit))
                    
        
        title_name ='\n' + error.upper() + ' PLOT \n'
        plt.title(title_name)
        plt.xlabel("Número de árvores")
        plt.ylabel(error)
        plt.legend();

## 1.4. Dataset

### 1.4.1. Descrição de dados

O dataset disponibilizado pelo kaggle, tem a finalidade de prever as espécies de bacterias com base em repetidas medições com fragmentos de DNA, os dados foram coletados com espectrocopia de Raman que calcula o histograma de base ATATGGCCTT no trecho A2T4G2C2. 

Cada linha de dados contém um espectro de histogramas gerados por medições repetidas de uma amostra, cada linha contendo a saída de todas as 286 possibilidades de histograma.

Os dados (tanto de treinamento quanto de teste) também contêm erros de medição simulados (taxas variadas) para muitas das amostras, o que torna o problema mais desafiador.

### 1.4.2. Variável de alvo
A meta para a competição deste mês é uma variável multiclasse, temos 10 classes que são:

- Streptococcus_pyogenes,
- Salmonella_enterica,
- Enterococcus_hirar,
- Escherichia_coli
- Campylobacter_jejuni
- Streptococcus_pneumoniae
- Staphylococcus_aureus'
- Escherichia_fergusonii'
- Bacteroides_fragilis'
- Klebsiella_pneumoniae



### 1.4.3. Arquivos
- **train.csv** - o conjunto de treinamento
- **test.csv** - o conjunto de teste; a tarefa é prever as espécies de bactérias (target) para cada row_id
- **sample_submission.csv** - um arquivo de envio de amostra no formato correto


### 1.4.4. Estrutura de pasta

In [ ]:
paths = ['img', 'Data', 'Data/pkl', 'Data/submission', 'Data/tunning', 
         'model', 'model/preds', 'model/optuna','model/preds/test', 
         'model/preds/test/n1', 'model/preds/test/n2', 'model/preds/test/n3', 
         'model/preds/train', 'model/preds/train/n1', 'model/preds/train/n2', 
         'model/preds/train/n3', 'model/preds/param']

for path in paths:
    try:
        os.mkdir(path)       
    except:
        pass  

### 1.4.5. Carrega dados

In [ ]:
path      = '/content/drive/MyDrive/kaggle/Tabular Playground Series/2022/02 - Fevereiro/'
path     = '../input/tabular-playground-series-feb-2022/'
path_data = ''  
target    = 'target'

In [ ]:
df1_train     = pd.read_csv(path + path_data + 'train.csv')
df1_test      = pd.read_csv(path + path_data + 'test.csv')
df_submission = pd.read_csv(path + path_data + 'sample_submission.csv')

df1_train.drop('row_id', axis=1, inplace=True)
df1_test.drop('row_id', axis=1, inplace=True)

df1_train.shape, df1_test.shape, df_submission.shape

In [ ]:
df1_train.head()

In [ ]:
df1_test.head()

### 1.4.6. Redução dos datasets

In [ ]:
df1_train = reduce_memory_usage(df1_train)
df1_test  = reduce_memory_usage(df1_test)

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>
Tivemos uma redução de aproximadamente de 75% em ambos os datasets.
    
</div>

# <div class="alert alert-success"> 2. Análise Exploratória de Dados (EDA)  </div> 

## 2.1. Dimensão do DataSet

In [ ]:
print('TREINO')
print('Number of Rows: {}'.format(df1_train.shape[0]))
print('Number of Columns: {}'.format(df1_train.shape[1]), end='\n\n')

print('TESTE')
print('Number of Rows: {}'.format(df1_test.shape[0]))
print('Number of Columns: {}'.format(df1_test.shape[1]))

## 2.2. Tipo de dados

In [ ]:
df1_train.info()

In [ ]:
df1_train.info()

In [ ]:
print(f'{3*"="} For Pandas {10*"="}\n{(df1_train.dtypes).value_counts()}')
print(f'\n{3*"="} For Datatable {7*"="}\n{(df1_test.dtypes).value_counts()}')

## 2.3. Identificar NA

In [ ]:
missing = missing_zero_values_table(df1_train)
missing[:].style.background_gradient(cmap='Reds')

In [ ]:
missing = missing_zero_values_table(df1_test)
missing[:].style.background_gradient(cmap='Reds')

<div class="alert alert-info" role="alert"> 
    
**`NOTA:`** <br>

Não temos dados faltantes.
    
</div>

## 2.4 Estatística Descritiva
Abaixo estão as estatísticas básicas para cada variável que contém informações sobre contagem, média, desvio padrão, mínimo, 1º quartil, mediana, 3º quartil e máximo.

In [ ]:
feature_float = []
feature_cat   = df1_test.select_dtypes(object).columns

for col in df1_test.columns:
    if col not in feature_cat: 
        feature_float.append(col)   

feature_float_test = feature_float.copy()

msg = 'Temos {} variávies numéricas e {} categóricas.'
print(msg.format(len(feature_float), len(feature_cat)))

### 2.4.1 Atributos numéricos

- Train

In [ ]:
df1_train[feature_float].describe().style.background_gradient(cmap='YlOrRd')

- Test

In [ ]:
df1_test[feature_float_test].describe().style.background_gradient(cmap='YlOrRd')

### 2.4.2. Atributos categóricos

In [ ]:
df1_train[feature_cat].columns

<div class="alert alert-info" role="alert"> 
    
**`NOTA:`** <br>
    
    
</div> 

## 2.5. Correlação
Vamos examinar a correlação entre as variáveis.

In [ ]:
graf_feature_corr(df=df1_train, annot_=False, threshold=.75, print_var=True)

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>
    
Como podemos observar, a correlação fica entre -0.4 e 0.80%, é um indicativo que temos variávies com alta correlação, neste caso temos 12 variáveis com correlação acima de 75%, na modelagem podemos fazer a exclusão de algumas dessas variáveis autocorrelacionadas.  
    
</div>

## 2.6. Distribuição

### 2.6.1. Train / Test

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

pie = ax.pie([len(df1_train), len(df1_test)],
             labels   = ["Train dataset", "Test dataset"],
             colors   = ["salmon", "teal"],
             textprops= {"fontsize": 15},
             autopct  = '%1.1f%%')

ax.axis("equal")
ax.set_title("Comparação de comprimento do conjunto de dados \n", fontsize=18)
fig.set_facecolor('white')
plt.show();

### 2.6.2. Proporção de variáveis

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

plt.pie([ len(feature_cat), len(feature_float)], 
        labels=['Categorical', 'Continuos' ],
        textprops={'fontsize': 13},
        autopct='%1.1f%%')

#ax.axis("equal")
ax.set_title("Comparação variáveis continuas/categóricas \n Dataset Treino/Teste", fontsize=18)
fig.set_facecolor('white')
plt.show()

### 2.6.3. Variável predidora

In [ ]:
plt.figure(figsize=(10, 5))
ax = sns.countplot(x=df1_train[target], palette='viridis')
ax.set_title('Distribuição da variável ' + target, fontsize=20, y=1.05)
ax.set_xticklabels(df1_train[target].unique(), rotation=90)
sns.despine(right=True)

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>
    
A análise do alvo não é particularmente reveladora,  por se tratar de um experimento, temos uma distribuição de alvos bastante equilibrada.
    
    
</div>

In [ ]:
stat = pd.DataFrame(df1_train[target].value_counts())
stat['percent_type'] = stat[target] / stat[target].sum()

stat.style.bar(subset=["percent_type"], color='yellow').format('{:.2%}', subset=["percent_type"])

### 2.6.4. Distribuição 

In [ ]:
fig, axs = plt.subplots(72, 4, figsize=(16,300))
           
for i, col in enumerate(feature_float):
    ax = axs.flat[i]
    ax.hist(np.log1p(df1_train[col]), bins=50, density=1, alpha=0.7, log=True)
    ax.set_title(col)       

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>
   
Podemos observar no gráfico acima, que os dados são muitos distorcidos, existem 286 histogramas,  vamos explorar melhor como eles estão distribuídos porque se pensarmos em todas as combinações possíveis de histogramas de ATGC com valores que variam de 0 a 11 o número deveria ter sido bem maior, cerca de 14641 histogramas. Os histogramas devem ser alguns selecionados por alguns critérios específicos. 
    
<br>

Primeiro vamos preparar uma maneira de converte as colunas em uma distribuição ATGC.    

<br>
**Referencia do código abaixo**: https://www.kaggle.com/lucamassaron/basic-eda-and-model-to-start, [@lucamassaron](https://www.kaggle.com/lucamassaron) 
    
</div>

In [ ]:
def split_feature(st):
    counts = list(map(int, re.split('A|T|G|C', st)[1:]))
    return counts

feature_atgc = {c: split_feature(c) for c in feature_float}

In [ ]:
# Referencia: https://www.kaggle.com/lucamassaron/basic-eda-and-model-to-start, @lucamassaron
a = [0 for i in range(11)]
t = [0 for i in range(11)]
g = [0 for i in range(11)]
c = [0 for i in range(11)]

grf_list = []

for feat in feature_float:
    xa, xt, xg, xc = feature_atgc[feat]
    a[xa] += 1
    t[xt] += 1
    g[xt] += 1
    c[xc] += 1

grf_list.append(a)
grf_list.append(t)
grf_list.append(g)
grf_list.append(c)

plt.figure(figsize=(20, 3))

for i , f in enumerate(['A','T','G','C']):
    plt.subplot(2, 2, i+1)
    plt.title(f)
    plt.bar(range(11), height=a)
    
plt.subplots_adjust(left    = 0.1,
                    bottom  = 0.1, 
                    right   = 0.7, 
                    top     = 1.3, 
                    wspace  = 0.3, 
                    hspace  = 0.5
                   )

plt.show()

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>
  
Nos gráficos acima, observamos claramente que a distribuição é desequilibrada, a qual não identificamos no primeiro gráfico, mas é igualmente para todas as letras, contagens mais baixas são mais frequentes do que as mais altas e a diminuição se ajusta a uma transformação de raiz quadrada.  
  
<br>
    
**OBSERVAÇÃO:** <br>
    
O espaço potencial para uma análise usando uma matriz multidimensional de 11x11x11x11 realmente não faz sentido porque você criaria uma estrutura de dados enorme para cada exemplo que tornaria muito esparsa, sendo assim,  tenho mais duas duas perguntas que tentarei responder:
    
<br>   
- existe uma maneira de representar o espaço dos histogramas melhor do que a forma proposta pela competição que poderia ser explorada pelo deep learning? (como uma sequência ou como um gráfico talvez) <br>

- Precisamos de todos os histogramas medidos? Não podemos fazer com muito menos?
    
<br>    
Vamos tentar visualizar cada alvo (target) que tem distribuição com cada variável, o gráfico pode não ser muito informativo por causa de muitas barras plotadas e podem apresentar possíveis dificuldades, pois esperamos que cada alvo tenha uma forma distinta, parcialmente mascarada pelo ruído de medição.
    
</div>

In [ ]:
# Referencia: https://www.kaggle.com/lucamassaron/basic-eda-and-model-to-start, @lucamassaron

df_freqs = df1_train.groupby('target').mean()

for tgt in df1_train.target.unique():
    fig, ax = plt.subplots(1, 1, figsize=(12,6))
    df_freqs[df_freqs.index==tgt].plot.bar(ax=ax, width=3, xlabel='')    
    _lg = ax.get_legend()
    _lg.remove()
    plt.xticks(rotation = 0)
    plt.title('Distribuição de medições')
    plt.show()

# <div class="alert alert-success"> 3. Modelagem (baseline) </div> 

## 3.1. Split Train/Test

In [ ]:
%%time
lb = LabelEncoder()

X      = df1_train.drop([target], axis=1)
y      = pd.DataFrame(lb.fit_transform(df1_train[target]), columns=[target])
X_test = df1_test

X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size    = 0.3,
                                                      shuffle      = True, 
                                                      stratify     = y, 
                                                      random_state = 12359)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape , X_test.shape

## 3.2. Parametros do modelo

In [ ]:
seed   = 12359
params = {"objective"     : 'multi:softmax',    
          'eval_metric'   : 'mlogloss',         
          'random_state'  : seed}

if torch.cuda.is_available():           
    params.update({'tree_method': 'gpu_hist','predictor': 'gpu_predictor'})
    
params

## 3.3. Seleção de Scaler

In [ ]:
%%time 

model_baseline = xgb.XGBClassifier(**params)

scalers = [None, 
           StandardScaler(), 
           RobustScaler(), 
           MinMaxScaler(), 
           MaxAbsScaler(), 
           QuantileTransformer(output_distribution='normal', random_state=0)
           ]

for scaler in scalers: 
    
    if scaler!=None:
        X_train_s = scaler.fit_transform(X_train)
        X_valid_s = scaler.transform(X_valid)
    else:
        X_train_s = X_train
        X_valid_s = X_valid
                
    model_baseline.fit(X_train_s, y_train, verbose=False)

    y_pred = model_baseline.predict(X_valid_s)  
    acc    = metrics.accuracy_score(y_valid, y_pred)    
        
    print('Validaçao ACC: {:2.5f} => {}'.format(acc, scaler))

    free_gpu_cache()

print()

Vamos fazer a predição em todos os dados e gerar os aquivos de submissões para cada scaler:

In [ ]:
%%time
path    = ''
scalers = [StandardScaler(), 
           RobustScaler(), 
           MinMaxScaler(), 
           MaxAbsScaler(), 
           QuantileTransformer(output_distribution='normal', random_state=0)]

for i, scaler in enumerate(scalers):    
    X_sc      = scaler.fit_transform(X)
    X_test_sc = scaler.transform(X_test)   
    
    model_baseline.fit(X_sc, y)    
    
    file_name = path + 'Data/submission/xgb_base_line_'+str(i+1)+'_'+str(scaler).lower()[:4]+'.csv'    
    df_submission[target] = lb.inverse_transform(model_baseline.predict(X_test_sc))
    df_submission.to_csv(file_name, index=False)

In [ ]:
(1- 0.90718/ 0.98258)*100

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>
    
Com **MaxAbsScaler** obtivemos uma ACC de 0.90718 na submissão do kaggle, abaixo o resultado de todas as submissões: <br>
  
- StandardScaler = 0.90452 <br>
- RobustScaler   = 0.90276 <br>
- MinMaxScaler   = 0.90351 <br>
- `MaxAbsScaler   = 0.90718`<br>
- QuantileTransformer = 0.90321
    
<br>

Como podemos observar nos resultados, o nosso modelo está em média 7.67% acima dos resultados das submissões do kaggle, é um indicativo que o modelo estar com overfitting, vamos treinar o modelo com uma validação cruzada, para termos uma estimativa mais robusta. 
    
</div>

## 3.4. Validação Cruzada

In [ ]:
def cross_val_model(model_, model_name_, X_, y_, X_test_, target_, scalers_, lb_, fold_=5, path_='',  
                    seed_=12359, feature_scaler_=None, print_report_=False):
    
    n_estimators = model_.get_params()['n_estimators']
             
    valid_preds     = {}
    taco            = 76 
    acc_best        = 0
    col_prob        = y_[target_].sort_values().unique()
    df_proba        = pd.DataFrame()
    feature_imp     = pd.DataFrame()
    test_preds      = []
    test_pred_proba = np.zeros((1, 1))
    preds           = []
    model           = []

    for i, scaler_ in enumerate(scalers_): 

        time_start   = datetime.now()
        score        = []        
                
        if scaler_!=None:            
            string_scaler = str(scaler_)        
            string_scaler = string_scaler[:string_scaler.index('(')]
        else:
            string_scaler = None 
            
        y_pred_test = np.zeros(len(X_test_))

        folds = KFold(n_splits=fold_, shuffle=True, random_state=seed_)
        folds = StratifiedKFold(n_splits=fold_, shuffle=True, random_state=seed_)
        
        print('='*taco)
        print('Scaler: {} - n_estimators: {}'.format(string_scaler, n_estimators))
        print('='*taco)
        
        for fold, (trn_idx, val_idx) in enumerate(folds.split(X_, y_, groups=y_)): 

            time_fold_start = datetime.now()

            # ---------------------------------------------------- 
            # Separar dados para treino 
            # ----------------------------------------------------     
            X_trn, X_val = X_.iloc[trn_idx], X_.iloc[val_idx]
            y_trn, y_val = y_.iloc[trn_idx], y_.iloc[val_idx] 
            
            # ---------------------------------------------------- 
            # Processamento 
            # ----------------------------------------------------     
            if scaler_!=None: 
                X_tst = X_test_.copy()
                if feature_scaler_!=None:                     
                    X_trn[feature_scaler_] = scaler_.fit_transform(X_trn[feature_scaler_])
                    X_val[feature_scaler_] = scaler_.transform(X_val[feature_scaler_])                      
                    X_tst[feature_scaler_] = scaler_.transform(X_tst[feature_scaler_])
                else:            
                    X_trn = scaler_.fit_transform(X_trn)
                    X_val = scaler_.transform(X_val)
                    X_tst = scaler_.transform(X_test_[feature_scaler_].copy())
                
            # ---------------------------------------------------- 
            # Treinar o modelo 
            # ----------------------------------------------------            
            model_.fit(X_trn, y_trn,
                       eval_set              = [(X_trn, y_trn), (X_val, y_val)],          
                       early_stopping_rounds = int(n_estimators*.1),
                       verbose               = False)
            
            # ---------------------------------------------------- 
            # Predição 
            # ----------------------------------------------------     
            y_pred_val      = model_.predict(X_val, ntree_limit=model_.best_ntree_limit)    
            y_pred_val_prob = model_.predict_proba(X_val, ntree_limit=model_.best_ntree_limit)    
            
            preds.append(model_.predict(X_tst))            
    
            df_prob_temp    = pd.DataFrame(y_pred_val_prob, columns=col_prob)
            y_pred_pbro_max = df_prob_temp.max(axis=1)

            df_prob_temp['fold']    = fold+1
            df_prob_temp['id']      = val_idx
            
            df_prob_temp['y_val']   = y_val.values
            df_prob_temp['y_pred']  = y_pred_val            
            df_prob_temp['y_proba'] = np.max(y_pred_val_prob, axis=1)
            df_prob_temp['scaler']  = str(string_scaler)
                        
            # ---------------------------------------------------- 
            # Score 
            # ---------------------------------------------------- 
            acc   = metrics.accuracy_score(y_val, y_pred_val)
            f1    = metrics.f1_score(y_val, y_pred_val, average='weighted') # average='micro'
            prec  = metrics.precision_score(y_val, y_pred_val, average='macro')
            
            score.append(acc)     
            
            # ---------------------------------------------------- 
            # Feature Importance
            # ----------------------------------------------------             
            feat_imp = pd.DataFrame(index   = X_.columns,
                                    data    = model_.feature_importances_,
                                    columns = ['fold_{}'.format(fold+1)])

            feat_imp['acc_'+str(fold+1)] = acc
            feature_imp = pd.concat([feature_imp, feat_imp], axis=1)
            
            # ---------------------------------------------------- 
            # Print resultado  
            # ---------------------------------------------------- 
            time_fold_end = diff(time_fold_start, datetime.now())
            msg = '[Fold {}] ACC: {:2.5f} - F1-macro: {:2.5f} - Precision: {:2.5f}  - {}'
            print(msg.format(fold+1, acc, f1, prec, time_fold_end))
            
            # ---------------------------------------------------- 
            # Salvar o modelo 
            # ---------------------------------------------------- 
            dic_model = {'scaler' : scaler_, 
                         'fold'   : fold+1, 
                         'model'  : model_ 
                          }
            
            model.append(dic_model)

        acc_mean = np.mean(score) 
        acc_std  = np.std(score)

        if acc_mean > acc_best:     
            acc_best    = acc_mean           
            model_best  = model_    
            scaler_best = scaler_

        time_end = diff(time_start, datetime.now())   

        print('-'*taco)
        print('[Mean Fold] ACC: {:2.5f} std: {:2.5f} - {}'.format(acc_mean, acc_std, time_end))
        print('='*taco)
        print()
                 
        df_submission[target_] = lb_.inverse_transform(mode(preds).mode[0]) # Majority vote       
        name_file_sub = model_name_ + '_' + str(i+1) + '_' + str(scaler_).lower()[:4] + '.csv'
        df_submission.to_csv(path_ + 'Data/submission/' + name_file_sub.format(acc_mean), index = False)

        if print_report_:
            y_pred = df_prob_temp[df_prob_temp['scaler']==str(string_scaler)]['y_pred']
            y_vl   = df_prob_temp[df_prob_temp['scaler']==str(string_scaler)]['y_val']
            print(metrics.classification_report(y_vl,y_pred))

    print('-'*taco)
    print('Scaler Best: {}'.format(scaler_best))
    print('Score      : {:2.5f}'.format(acc_best))
    print('-'*taco)
    print()

    return model , df_prob_temp.sort_values(by=['scaler','id']) , feature_imp

In [ ]:
%%time 
scalers = [StandardScaler(), 
           RobustScaler(), 
           MinMaxScaler(), 
           MaxAbsScaler(), 
           QuantileTransformer(output_distribution='normal', random_state=0)
          ]

seed     = 12359
                 
eval_metric = ['mlogloss','merror']
        
params = {"objective"     : 'multi:softmax',    
          'eval_metric'   : 'mlogloss',         
          'random_state'  : seed, 
          'n_jobs'        : -1}

if torch.cuda.is_available():           
    params.update({'tree_method' : 'gpu_hist',                    
                   'predictor'   : 'gpu_predictor'})

model, df_proba, feature_imp = cross_val_model(model_         = xgb.XGBClassifier(**params),
                                               model_name_    = 'xgb_bs_vc_score_01_{:2.5f}',
                                               X_             = X,
                                               y_             = y,
                                               X_test_        = X_test,
                                               target_        = target,
                                               scalers_       = scalers,
                                               fold_          = 5, 
                                               lb_            = lb,
                                               path_          = '',
                                               seed_          = seed, 
                                               feature_scaler_= feature_float_test, 
                                               print_report_  = False)

print()

<div class="alert alert-info" role="alert"> 
    
**`NOTA:`** <br>
 
Com **RobustScale** obtivemos uma ACC de 0.94297 na submissão do kaggle, abaixo o resultado de todas as submissões: <br>
    
- StandardScaler = 0.94006 <br>
- __RobustScaler  = 0.94297__ <br>
- MinMaxScaler  = 0.94101 <br>
- MaxAbsScaler  = 0.94187 <br>
- QuantileTransformer = 0.94146 <br>
    
<br>
    
Como podemos observar nos resultados, o nosso modelo estar em média 4.22% acima dos resultados das submissões do kaggle, porém com uma redução de erro de 44.98% em relação ao modelo anterior, que foi treinado com todos os dados. O ideal é que tenhamos um erro em torno de +-2% das previsões geradas pelos modelos e as submissões no kaggle, pois temos que levar em consideração que nossas submissiões são avaliadas em 20% dos dados de treino. 
    
</div>

## 3.5. Análise do Modelo 
Vamos fazer o treinamento novamente do melhor modelo com 1000 estimadores e fazer a previsão utilizando o limite ideal sugerido pelo atributo `best_ntree_limit` do XGBoost.


In [ ]:
%%time 

scalers     = [RobustScaler()]
eval_metric = ['mlogloss','merror']

params = {"objective"     : 'multi:softmax',    
          'eval_metric'   : eval_metric,  
          'n_estimators'  : 1000,
          'random_state'  : seed, 
          'n_jobs'        : -1}

if torch.cuda.is_available():           
    params.update({'tree_method' : 'gpu_hist',                    
                   'predictor'   : 'gpu_predictor'})
    
model, df_proba, feature_imp = cross_val_model(model_         = xgb.XGBClassifier(**params),
                                               model_name_    = '_xgb_bl_cv_score_02_{:2.5f}',
                                               X_             = X,
                                               y_             = y,
                                               X_test_        = X_test,
                                               target_        = target,
                                               scalers_       = scalers,
                                               fold_          = 5, 
                                               lb_            = lb,
                                               path_          = '',
                                               seed_          = seed, 
                                               feature_scaler_= feature_float_test, 
                                               print_report_  = True)
print()

<div class="alert alert-info" role="alert"> 
    
**`NOTA:`** <br>
    
Com o acrescimo de 1000 estimadores o modelo obteve uma ACC de 0.93920, sendo inferior a submissão anterior.      

           
</div>

### 3.5.1. Feature Importances  

In [ ]:

plt.figure(figsize=(20,12))
for fold, col in enumerate(feature_imp.filter(regex=r'fold').columns):            
    col_acc = 'acc_' + str(fold+1)
    df_fi = feature_imp.sort_values(by=col, ascending=False).reset_index().iloc[:25]
    df_fi = df_fi[['index', col, col_acc]]
    df_fi.columns = ['Feature', 'score', col_acc]
    plt.subplot(2,3, fold+1)
    sns.barplot(x='score', y='Feature', data=df_fi)    
    plt.title('Fold {} - score: {:2.5f}'.format(fold+1, df_fi[col_acc].mean()), 
              fontdict={'fontsize':18})    

plt.suptitle('Feature Importance XGB', y=1.05, fontsize=24);
plt.tight_layout(h_pad=3.0); 

<div class="alert alert-info" role="alert"> 
    
**`NOTA:`** <br>
    
    
</div>

### 3.1.1. Erro e  número de Estimadores

In [ ]:
for erro in eval_metric:
    plt.figure(figsize=(20,12))

    for i in range(len(model)):
        results     = model[i]['model'].evals_result() # merror
        ntree_limit = model[i]['model'].best_ntree_limit

        plt.subplot(2,3,i+1)
        plt.plot(results["validation_0"][erro], label="Treinamento")
        plt.plot(results["validation_1"][erro], label="Validação")

        plt.axvline(ntree_limit, 
                    color="gray", 
                    label="N. de árvore ideal {}".format(ntree_limit))

        plt.xlabel("Número de árvores")
        plt.ylabel(erro)
        plt.legend();

    plt.suptitle('Performance XGB - {}'.format(erro), y=1.05, fontsize=24);
    plt.tight_layout(h_pad=3.0);

<div class="alert alert-info" role="alert"> 
    
**`NOTA:`** <br>
Acima recuperamos as informações de treinamento do nosso modelo, podemos observar que o número de 1000 estimadores é mais que suficiente para o treinamento do modelo, o ideal é que fique em entre de 300 à 350 para esses dados e com a utilização dos parametros padrões que devem ser ajustados para o `XGBoost`. <br>
  

Os próximos passos são: <br>
- Feature Engineering
- Tuning de hyperparametros    
    
</div>